In [27]:
import os
import nltk
import numpy as np
import copy
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torch.utils.tensorboard
from torch.utils.tensorboard import SummaryWriter
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [28]:
nltk.download('punkt')

documents = {}
labels = {}

# open main data
for file in os.listdir("/home/valeriya/Desktop/UMD/Computational_linguistic/Project/Hulth2003/Hulth2003/Training"):
    if file.endswith(".abstr"):
        content = open(("%s/%s" % ('/home/valeriya/Desktop/UMD/Computational_linguistic/Project/Hulth2003/Hulth2003/Training', file)), "r").read()
        documents[file.split('.')[0]] = content.split('. ')
        
# open labels        
for file in os.listdir("/home/valeriya/Desktop/UMD/Computational_linguistic/Project/Hulth2003/Hulth2003/Training"):
    if file.endswith(".uncontr"):
        content = open(("%s/%s" % ('/home/valeriya/Desktop/UMD/Computational_linguistic/Project/Hulth2003/Hulth2003/Training', file)), "r").read()
        labels[file.split('.')[0]] = content.split("; ")       

[nltk_data] Downloading package punkt to /home/valeriya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
# tokenize document
tokenized_documents = {}
for num, ctt in documents.items():
    tokenized_documents[num] = []
    for sentence in ctt:
        words = nltk.word_tokenize(sentence.lower())
        #words=[word.lower() for word in words if word.isalpha()]
        tokenized_documents[num].append(words)

# tokenize labels
tokenized_labels = {}
for num, ctt in labels.items():
    tokenized_labels[num] = []
    for sentence in ctt:
        words = nltk.word_tokenize(sentence.lower())
        #words=[word.lower() for word in words if word.isalpha()]
        tokenized_labels[num].append(words)        

In [30]:
#PROBLEM 
# For document 1346 keyphrase should be "growing time adaptive self-organizing map", but word self-organized is lost
# Fixed

In [31]:
tokenized_labels['1346']

[['automatic', 'multilevel', 'thresholding'],
 ['image', 'segmentation'],
 ['growing', 'time', 'adaptive', 'self-organizing', 'map'],
 ['growing', 'tasom'],
 ['gtasom'],
 ['peak', 'finding', 'process']]

In [32]:
# function find index of element in list of lists
def index(lab, target):
    for i,phr in enumerate(lab):
        for j, w in enumerate(phr):
            if w == target:
                return (j)
    return (None, None)

# create dictionary of labels associated to words
class_labels = copy.deepcopy(tokenized_documents)
for document in tokenized_documents:
    text = tokenized_documents[document]
    lab = tokenized_labels[document]
    lab_flattened = [val for sublist in lab for val in sublist]
    for i, sentence in enumerate(text): 
        for j, word in enumerate(sentence): 
            is_keyphrase = word in lab_flattened
            if is_keyphrase:
                if index(tokenized_labels[document], word) == 0:
                    class_labels[document][i][j] = "first"
                else:
                    class_labels[document][i][j] = 'inside'
            else:
                class_labels[document][i][j] = 'no'

In [33]:
tokenized_documents['761']

[['towards',
  'a',
  'nmr',
  'implementation',
  'of',
  'a',
  'quantum',
  'lattice',
  'gas',
  'algorithm',
  'recent',
  'theoretical',
  'results',
  'suggest',
  'that',
  'an',
  'array',
  'of',
  'quantum',
  'information',
  'processors',
  'communicating',
  'via',
  'classical',
  'channels',
  'can',
  'be',
  'used',
  'to',
  'solve',
  'fluid',
  'dynamics',
  'problems'],
 ['quantum',
  'lattice-gas',
  'algorithms',
  '(',
  'qlga',
  ')',
  'running',
  'on',
  'such',
  'architectures',
  'have',
  'been',
  'shown',
  'to',
  'solve',
  'the',
  'diffusion',
  'equation',
  'and',
  'the',
  'nonlinear',
  'burgers',
  'equations'],
 ['in',
  'this',
  'report',
  ',',
  'we',
  'describe',
  'progress',
  'towards',
  'an',
  'ensemble',
  'nuclear',
  'magnetic',
  'resonance',
  '(',
  'nmr',
  ')',
  'implementation',
  'of',
  'a',
  'qlga',
  'that',
  'solves',
  'the',
  'diffusion',
  'equation'],
 ['the',
  'methods',
  'rely',
  'on',
  'nmr',
  'tech

In [34]:
tokenized_labels['761']

[['nmr', 'implementation'],
 ['quantum', 'lattice', 'gas', 'algorithm'],
 ['quantum', 'information', 'processors'],
 ['fluid', 'dynamics', 'problems'],
 ['diffusion', 'equation'],
 ['nonlinear', 'burgers', 'equations'],
 ['nuclear', 'magnetic', 'resonance'],
 ['two-qubit', 'quantum', 'information.processors']]

In [35]:
class_labels['761']

[['no',
  'no',
  'first',
  'inside',
  'no',
  'no',
  'first',
  'inside',
  'inside',
  'inside',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'first',
  'inside',
  'inside',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'first',
  'inside',
  'inside'],
 ['first',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'first',
  'inside',
  'no',
  'no',
  'first',
  'inside',
  'inside'],
 ['no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'first',
  'inside',
  'inside',
  'no',
  'first',
  'no',
  'inside',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'first',
  'inside'],
 ['no',
  'no',
  'no',
  'no',
  'first',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'first',
  'first',
  'inside',
  'inside'],
 ['no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
  'no',
 

# GLOVE embeddings

In [36]:
import numpy as np
embeddings = dict()
embed_size = 100
f = open('glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings[word] = coefs
f.close()

# Data padding

In [37]:
# for data we use sentences individually, not documents
copy.deepcopy(tokenized_documents).values()
X = [sent for doc in copy.deepcopy(tokenized_documents).values() for sent in doc]
y = [sent for doc in copy.deepcopy(class_labels).values() for sent in doc]

# padding the data
def Padding(data):
    len_max = len(max(X, key=lambda coll: len(coll)))
    for i, sentence in enumerate(data):
        len_sent = len(sentence)
        len_pad = len_max - len_sent
        sentence.extend(['PAD' for i in range(len_pad)])
    return(data)

X_padded = Padding(X)
y_padded = Padding(y)
X_lengths = len(X_padded[0])

In [38]:
embeddings['PAD'] = 0

# BiLSTM Net

In [52]:
class LSTM(nn.Module):
    def __init__(self, nb_layers = 1, nb_lstm_units=150, embedding_dim=100, batch_size=1):
        super(LSTM, self).__init__()
        self.vocab = embeddings
        self.tags = {'PAD': 0, 'first': 1, 'inside': 2, 'no': 3}

        self.nb_lstm_layers = nb_layers
        self.nb_lstm_units = nb_lstm_units
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size

        # don't count the padding tag for the classifier output
        self.nb_tags = len(self.tags) - 1


        nb_vocab_words = len(self.vocab)

        # whenever the embedding sees the padding index it'll make the whole vector zeros
        padding_idx = self.vocab['PAD']
        print(padding_idx)
        self.word_embedding = nn.Embedding(
            num_embeddings=nb_vocab_words,
            embedding_dim=self.embedding_dim,
            padding_idx=padding_idx
        )

        # design LSTM
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.nb_lstm_units,
            num_layers=self.nb_lstm_layers,
            batch_first=True,
        )

        # output layer which projects back to tag space
        self.lin1 = nn.Linear(self.nb_lstm_units, nb_lstm_units)
        self.lin2 = nn.Linear(self.nb_lstm_units, self.nb_tags)

    def init_hidden(self):
        # the weights are of the form (nb_layers, batch_size, nb_lstm_units)
        hidden_a = torch.randn(self.hparams.nb_lstm_layers, self.batch_size, self.nb_lstm_units).to(device)
        hidden_b = torch.randn(self.hparams.nb_lstm_layers, self.batch_size, self.nb_lstm_units).to(device)
        hidden_a = Variable(hidden_a)
        hidden_b = Variable(hidden_b)

        return (hidden_a, hidden_b)

    def forward(self, X, X_lengths):
        # reset the LSTM hidden state. Must be done before you run a new batch. Otherwise the LSTM will treat
        # a new batch as a continuation of a sequence
        self.hidden = self.init_hidden()
        batch_size, seq_len, _ = X.size()

        # 1. embed the input
        X = self.word_embedding(X)

        # 2. Run through RNN
        # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        X = torch.nn.utils.rnn.pack_padded_sequence(x, X_lengths, batch_first=True)

        # now run through LSTM
        X, self.hidden = self.lstm(X, self.hidden)

        # undo the packing operation
        X, _ = torch.nn.utils.rnn.pad_packed_sequence(X, batch_first=True)

        # 3. Project to tag space
        # this one is a bit tricky as well. First we need to reshape the data so it goes into the linear layer
        X = X.contiguous()
        X = X.view(-1, X.shape[2])

        # run through actual linear layer
        X = F.relu(self.lin1(X))
        X = F.relu(self.lin2(X))

        # I like to reshape for mental sanity so we're back to (batch_size, seq_len, nb_tags)
        X = X.view(batch_size, seq_len, self.nb_tags)

        Y_hat = X
        return Y_hat

In [53]:
net = LSTM()
criterion= torch.nn.CrossEntropyLoss(size_average=True, ignore_index=0)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

0


/home/valeriya/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


# Here we need function computing batches

In [54]:
def train(epoch):
    train_loss = 0.0
    total = 0
    correct = 0
    for i, data in enumerate(X_padded):
        # get the inputs; data is a list of [inputs, labels]
        print(y_padded[i])
        labels = torch.Tensor(y_padded[i]).to('device')
        data = torch.Tensor(data).to('device')
        
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        _, outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # statistics
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    acc = 100.*correct/total
    print('Train accuracy:', acc)
    tb.add_scalar('Loss', train_loss, epoch)
    tb.add_scalar('Accuracy', acc, epoch)
    return(train_loss)



In [55]:
for epoch in range(1):
    train_loss = train(epoch)
    print(train_loss)

['first', 'inside', 'inside', 'no', 'first', 'inside', 'no', 'no', 'first', 'inside', 'inside', 'inside', 'inside', 'no', 'no', 'no', 'no', 'no', 'first', 'inside', 'no', 'inside', 'inside', 'inside', 'inside', 'no', 'no', 'no', 'no', 'first', 'no', 'no', 'no', 'no', 'first', 'inside', 'inside', 'no', 'no', 'no', 'first', 'inside', 'inside', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']


ValueError: too many dimensions 'str'